In [1]:
#to run a server, config allows us to change the general look, allows use of CSS
from pywebio import start_server, config
#allows input fields to remain on page after submission (Makes fields persistent)
from pywebio.pin import *
#allows use of input commands
from pywebio.input import *
#allows use of output commands
from pywebio.output import *
from functools import partial
import Database as AccessDB


In [2]:
#Declaring & initializing global styles as string of CSS code to keep consistent formatting throughout
headerStyle = 'text-align: center; background-color: tan; padding-top: 40px; padding-bottom: 40px; color: brown; font-size: 50px; font-weight: bold'
navButtonStyle = 'text-align: left; margin-top: 20px'
headingStyle = 'text-align: center; color:black; font-size: xx-large; font-weight: bold'
promptStyle = 'text-align: center; color:black; font-size: x-large; font-weight: bold'
headingUnderlineStyle = 'color: black; font-size: 15pt; font-weight: bold; text-decoration-line: underline'

recipeHeadingStyle = 'margin-left: 400px; color: black; font-size: 40pt; font-weight: bold'
recipeDescriptionStyle = 'font-size: 20pt; text-align: center'
recipeStatsStyle = 'font-size: 20pt; text-align: center; border: black'
recipeIngredStyle = 'padding-left: 100px; font-size 30pt'
recipeDirectionStyle = 'padding-left: 100px; font-size = 35pt; font-weight: bold'

centerStyle = 'text-align: center'
rightStyle = 'text-align: right'
leftStyle = 'text-align: left'

buttonStyle = ''

global pin


In [3]:
###Start Function### 
def start_app(): 
    AccessDB.open_cookbook() 
    main_menu() 

In [4]:
###VIEW FUNCTIONS### 
def print_header(): 
    ###Function to print running header & nav bar###    
    # # Nav bar button to return to main menu  
    # put_button(["Main Menu"], onclick=main_menu).style('text-align: left; margin-top: 20px') 
    # App title header 
    put_markdown('# The Busy Chef').style(headerStyle) 

In [5]:
def main_menu(): 
    ###Layout for main menu### 
    #Clear any prior output 
    clear() 
    with use_scope("main_menu", clear = True): 
        #display main menu header at top of page 
        put_markdown("# Welcome to the Busy Chef").style(headerStyle) 
        #Prompt user to make selection 
        put_text("What would you like to do?").style(promptStyle) 
        #Define & layout buttons for options 
        put_buttons(["View Recipes", 'Add Recipe'], onclick=menuButtons).style('text-align: center') 

def select_recipe(): 
    ###Layout & data for View All Recipes screen### 
    #Clear any prior output & display common header at top of page 
    clear() 
    print_header() 
    put_text("Recipes").style(recipeHeadingStyle) 
    #Get list of recipe names from DB 
    optionsList = AccessDB.getRecipeList() 
    #Populate DDL
    response = input_group("Please Enter your Recipe Information",[ 
        select('Which recipe would you like to view?', options = optionsList, name  = 'Recipe') 
    ], cancelable=True)
    #response = input_group("Recipe Selection",[select('Which recipe would you like to view?', optionsList), input('Enter Recipe Description', name = 'Description', type = TEXT)], cancelable=True)
    if (response != None):
        display_recipe(response['Recipe']) 
    else:
        main_menu()
def display_recipe(response): 

    ###Get data for recipe and format & display it to page### 
    #Clear any prior output & display common header at top of page 
    clear() 
    print_header() 
    #Get data for recipe 
    recipe = AccessDB.getRecipeInformation(response) 

    #Navigation Buttons
    put_buttons(["Main Menu", "Recipe Selection"], onclick=partial(recipeButtons, recipe = recipe['RecipeName'])).style(leftStyle), 

    #format & display recipe 
    #Recipe Heading: Name & Delete button 
    put_row([
        put_text(""), 
        put_text(recipe['RecipeName']).style(headingStyle),
        put_buttons(['Delete Recipe'], onclick=partial(recipeButtons, recipe = recipe['RecipeName'])).style(rightStyle)
        ])
    #Editing Buttons
    put_buttons(["Edit Details", "Edit Ingredients","Edit Directions"], onclick=partial(recipeButtons, recipe = recipe['RecipeName'])).style(centerStyle)
    
    #Recipe Description 
    put_text(recipe['Description']).style('recipeDescriptionStyle') 
    #Recipe General Info/Stats
    #Ingredients 
    ingredients = []
    for ingredient in recipe['ingredients']:
        amount = "{}\t{}".format(ingredient['Amount'], ingredient['Unit'])
        ingredients.append({'Ingredient': ingredient['Name'], 'Amount': amount})

    put_row([
        put_text(""), 
        put_table(ingredients).style(centerStyle),
        put_text("")
        ])
    put_text('Directions').style(headingUnderlineStyle)
    steps = []
    stepNum = 1
    for step in recipe['instructions']:
        txt = "{}. {}".format(stepNum,step)
        steps.append(txt)
        put_text(txt)
        stepNum += 1
    
def add_recipe(): 
    ###Layout for user input screen to gather new recipe information### 
    #Clear any prior output & display common header at top of page 
    clear() 
    print_header()
    #get information from user 
    info = input_group("Please Enter your Recipe Information",[ 
        input('Enter Recipe Name', name = 'RecipeName', type = TEXT, required=True), 
        input('Enter Recipe Description', name = 'Description', type = TEXT), 
        radio('Select your Tags', options = ['Vegetarian', 'Heart Healthy', 'Protien Heavy', 'Party'], name = 'tags'),       
        input('Food Category (Example: Indian, American, Italian)', name = 'foodCat', type = TEXT), 
        select('Cuisine', options = ['Breakfast', 'Brunch', 'Lunch', 'Dinner', 'Appetizer', 'Dessert'], name  = 'cuisine'), 
        input('Enter the Prep Time for the Meal', name = 'prepTime', type = NUMBER), 
        input('Enter the Cook Time for the Meal', name = 'cookTime', type = NUMBER), 
        input('Enter the Number of Servings for the Meal', name = 'servings', type = NUMBER),
        input('Enter the number of ingredients in recipe:', name = 'numIngred', type = NUMBER, required=True), 
        input('Enter the number of steps in recipe:', name = 'numSteps', type = NUMBER, required=True) 
    ], cancelable=True)

    if (info != None):
        #Get each ingredient info 
        #List to store each step entry
        ingredients = []
        #Loop to output input field for ingredients to page & collect input
        for i in range(info['numIngred']): 
            ingred = get_ingredients(i) 
            ingredients.append(ingred) 
        #Add final list of ingredients to recipe info dictionary
        info.update({'ingredients':ingredients})
        
        #Get each step info 
        #List to store each step entry
        steps = []
        #Loop to output input field for instructions to page & collect input
        for i in range(info['numSteps']): 
            step = get_steps(i) 
            steps.append(step['Instruction'])
        #Add final list of steps to recipe info dictionary
        info.update({'instructions':steps})

        ##Send recipe data to DBmodel to save in DB
        AccessDB.addRecipe(info)
    ###Go to main menu after submission or canceling### 
    main_menu() 

In [6]:

def get_ingredients(num): 
    ###Layout for user input screens to gather new recipe ingredients### 
    #Clear any prior output & display common header at top of page 
    clear() 
    print_header() 
    #String for recipe ingredient prompt with automatic incremented ingredient number 
    inputStr = "Enter Ingredient Information for Ingredient {}".format(num + 1) 
    #Displays input field & prompt for recipe ingredient, amount & measurement 
    ingred_info = input_group(inputStr, [ 

        input('Ingredient Name', name = 'Name', type = TEXT, required=True), 

        input('Ingredient Amount', name = 'Amount', type = TEXT, required=True), 

        input('Ingredient Measurement(example: cup, teaspoon)', name ='Unit', type = TEXT, required=True) 
    ]) 
    return ingred_info 

def get_steps(num): 

    ###Layout for user input screens to gather new recipe steps### 
    #Clear any prior output & display common header at top of page 
    clear() 
    print_header() 
    #String for recipe step prompt with automatic incremented step number 
    inputStr = "Enter Step {}".format(num + 1) 
    #Displays input field & prompt for recipe step  
    info = input_group(inputStr, [ 

        input('Instruction', name = 'Instruction', type = TEXT, required=True) 

    ]) 
    step = num + 1 
    step = str(step) + '.'  
    return info 
ingredNum = 0

In [7]:
###CONTROLS###
def menuButtons(btn_val): 
    ###Tells main menu buttons what to do/what functions to call### 
    #Launches View Recipes Page 
    if btn_val == "View Recipes": 
        select_recipe() 
    #Launches Add recipe page 
    elif btn_val == "Add Recipe": 
        add_recipe() 

def recipeButtons(btn_val, recipe):
    if btn_val == 'Main Menu':
        main_menu()
    elif btn_val == 'Edit Details':
        editRecipe(recipe)
    elif btn_val == 'Edit Ingredients':
        editIngredients(recipe)
    elif btn_val == 'Edit Directions':
        editDirections(recipe)
    elif btn_val == 'Delete Recipe':
        deleteRecipe(recipe)
    elif btn_val == 'Recipe Selection':
        select_recipe()



In [8]:
def editRecipe(currentRecipeName):
    clear() 
    print_header()
    #Get current recipe info to prepopulate form
    currentRecipe = AccessDB.getRecipeInformation(currentRecipeName)

    #Input fields prepopulated with current recipe that user can edit 
    info = input_group("Please Enter your Recipe Information",[ 
        input('Enter Recipe Name', name = 'RecipeName', type = TEXT, required=True, value=currentRecipe['RecipeName']), 
        input('Enter Recipe Description', name = 'Description', type = TEXT, value=currentRecipe['Description'] ),
        radio('Select your Tags', options = ['Vegetarian', 'Heart Healthy', 'Protien Heavy', 'Party'], name = 'tags', value=currentRecipe['tags'] ),       
        input('Food Category (Example: Indian, American, Italian)', name = 'foodCat', type = TEXT, value=currentRecipe['foodCat'] ), 
        select('Cuisine', options = ['Breakfast', 'Brunch', 'Lunch', 'Dinner', 'Appetizer', 'Dessert'], name  = 'cuisine', value=currentRecipe['cuisine'] ), 
        input('Enter the Prep Time for the Meal', name = 'prepTime', type = NUMBER, value=currentRecipe['prepTime'] ), 
        input('Enter the Cook Time for the Meal', name = 'cookTime', type = NUMBER, value=currentRecipe['cookTime'] ), 
        input('Enter the Number of Servings for the Meal', name = 'servings', type = NUMBER, value=currentRecipe['servings'] ),
        input('Enter the number of ingredients in recipe:', name = 'numIngred', type = NUMBER, required=True, value=currentRecipe['numIngred'] ), 
        input('Enter the number of steps in recipe:', name = 'numSteps', type = NUMBER, required=True, value=currentRecipe['numSteps'] ),
    ], cancelable=True)
    
    #If user doesn't click cancel
    if (info != None):
        #Update currentRecipe dictionary
        currentRecipe.update(info)
        #Call editRecipe function for model to update excel file
        #AccessDB.editRecipe(currentRecipeName, currentRecipe)

    #return to recipe when done
    display_recipe(currentRecipe['RecipeName'])

def editIngredients(recipeName):
    global pin #we are using globalized pin to get information from user

    clear()
    print_header()
    #function is inside editIngredients because it is only accessed within editIngredients function
    def __save_ingredients(ingredNum, recipeName):  #function will go through the pin information and save the latest data
        ingredients = []
        for i in range(1, ingredNum):
            nameIngredient = "{}{}".format("Ingredient",i)
            nameAmount = "{}{}".format("Amount",i)
            nameUnit = "{}{}".format("Unit",i)
            dictionary = {"Name": pin[nameIngredient], "Amount": pin[nameAmount], "Unit": pin[nameUnit]}
            ingredients.append(dictionary)
        AccessDB.editIngredients(ingredients, recipeName) #save the data
        display_recipe(recipeName) #go back to display the latest version of the recipe
        return #end of save_ingredients function
    
    #This section will display the ingredients to the user
    oldIngredients = AccessDB.getRecipeInformation(recipeName)
    oldIngredients = oldIngredients['ingredients'] #get old ingredients

    ingredients = [] #create a list of all of the current ingredients to display them for user
    for ingredient in oldIngredients:
        amount = "{}\t{}".format(ingredient['Amount'], ingredient['Unit'])
        ingredients.append({'Ingredient': ingredient['Name'], 'Amount': amount})
    put_row([
        put_text(""), 
        put_table(ingredients).style(centerStyle),
        put_text("")
        ])
    #end of section to display ingredients
    #Now we will put all of the input fields for the user to make changes to their existing recipe
    ingredNum = 1 #start at 1 for our ingredient number
    for ingredient in oldIngredients: # go through our existing ingredient information and put an input box for the user to edit their ingredient information
        nameIngredient = "{}{}".format("Ingredient",ingredNum)
        nameAmount = "{}{}".format("Amount",ingredNum)
        nameUnit = "{}{}".format("Unit",ingredNum)
        label = "Ingredient {}".format(ingredNum)
        put_text(label) 
        put_text("Ingredient Name:") #label for the input box
        put_input(nameIngredient, type='text', value=ingredient['Name']) #put input box with existing ingredient name
        put_text("Amount:") #label for input box
        put_input(nameAmount, type='number',value=ingredient['Amount']) #put input box with existing amount
        put_text("Unit:")# label for the input box
        put_input(nameUnit, type ='text',value=ingredient['Unit']) #put input box with existing unit 
        ingredNum +=1 #iterate up our ingredNum to go through and print every ingredient

    put_button(["Save"], onclick = partial(__save_ingredients, ingredNum = ingredNum, recipeName = recipeName)) #button for user to save their information
    
def editDirections(recipeName):
    global pin #we are using globalized pin to get information from user

    clear()
    print_header()
    #function is inside editIngredients because it is only accessed within editIngredients function
    def __save_steps(stepNum, recipeName):  #function will go through the pin information and save the latest data
        steps = []
        for i in range(1, stepNum):
            stepInput = "{}{}".format("Step",i)
            steps.append(pin[stepInput])
        AccessDB.editSteps(steps, recipeName) #save the data
        display_recipe(recipeName) #go back to display the latest version of the recipe
        return #end of save_steps function
    
    #This section will display the steps to the user
    oldSteps = AccessDB.getRecipeInformation(recipeName)
    oldSteps = oldSteps['instructions'] #get old steps
    put_text('Directions').style(headingUnderlineStyle)
    steps = []
    stepNum = 1
    for step in oldSteps:
        txt = "{}. {}".format(stepNum,step)
        steps.append(txt)
        put_text(txt)
        stepNum += 1
    #end of section to display steps

    #Now we will put all of the input fields for the user to make changes to their existing recipe
    stepNum = 1 #start at 1 for our step number
    for step in oldSteps: # go through our existing ingredient information and put an input box for the user to edit their ingredient information
        stepInput = "{}{}".format("Step",stepNum)
        label = "Step {}".format(stepNum)
        put_text(label) 
        put_input(stepInput, type ='text',value=step) #put input box with existing unit 
        stepNum +=1 #iterate up our ingredNum to go through and print every ingredient

    put_button(["Save"], onclick = partial(__save_steps, stepNum = stepNum, recipeName = recipeName)) #button for user to save their information
    return
    
def deleteRecipe(recipeName):
    #Pop out notification to avoid accidental deletions
    #Delete recipe from DB
    AccessDB.deleteRecipe(recipeName)
    #Return to main menu
    main_menu()
    return

In [9]:
#FIXME: Add save button
#FIXME: This will work for directions, but we need 3 fields for ingredients
#Function to add prefilled ingredient field
def add_ingred(currentValue):
    global ingredNum 
    ingredNum += 1
    lbl = "ingred{}".format(ingredNum)
    with use_scope('ingredients'):
        put_input(lbl, type='text')

#Function to add empty ingredient field
def add_ingred():
    global ingredNum 
    ingredNum += 1
    lbl = "ingred{}".format(ingredNum)
    with use_scope('ingredients'):
        put_input(lbl, type='text')
        
#Add empty field & button to add more        
def input_ingredients():
    add_ingred()
    put_button(["Add Ingredient"], onclick = add_ingred)

In [10]:
###PYWEBIO FUNCTION THAT OPENS UNUSED PORT ON NETWORK###
###Server starts with main_menu page### 
start_server(start_app, port = 8080, debug = True ) 

Running on all addresses.
Use http://192.168.1.216:8080/ to access the application


RuntimeError: This event loop is already running